In [1]:
!pip install -q Pillow
!pip install -q torch
!pip install -q torchvision
!pip install -q transformers
!pip install -q sentencepiece
!pip install -q accelerate
!pip install -q bitsandbytes
!pip install -q tqdm

In [2]:
import pandas as pd
import numpy as np
import os
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer
import re
from tqdm import tqdm
import requests

In [3]:
model_id = 'openbmb/MiniCPM-Llama3-V-2_5-int4'
model = AutoModel.from_pretrained(model_id , trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id , trust_remote_code=True)
model.eval()

config.json:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

configuration_minicpm.py:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-Llama3-V-2_5-int4:
- configuration_minicpm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_minicpmv.py:   0%|          | 0.00/25.0k [00:00<?, ?B/s]

resampler.py:   0%|          | 0.00/5.49k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

MiniCPMV(
  (llm): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(128256, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      

In [4]:
image_path = '/kaggle/working/img.jpg'

def download_image(url, image_path):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            with open(image_path, 'wb') as file:
                file.write(response.content)
            print(f"Downloaded: {image_path}")
        else:
            print(f"Failed to download: {url}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")


# download_image(df['image_link'][10], image_path)


In [5]:
data = pd.read_csv('/kaggle/input/ml-hackathon/test.csv')

print(data.shape)
data = data[9000:]

(131187, 4)


In [6]:
data.shape

(122187, 4)

In [7]:
data.head()

,index,image_link,group_id,entity_name
9000,9011,https://m.media-amazon.com/images/I/41PfRCmb8K...,648011,wattage
9001,9012,https://m.media-amazon.com/images/I/41PfRCmb8K...,648011,voltage
9002,9013,https://m.media-amazon.com/images/I/41Pg3Jbm2P...,847223,depth
9003,9014,https://m.media-amazon.com/images/I/41Pg3Jbm2P...,847223,width
9004,9015,https://m.media-amazon.com/images/I/41PgeeKqFZ...,525429,voltage


In [8]:
data.tail()

,index,image_link,group_id,entity_name
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight
131186,131287,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,maximum_weight_recommendation


In [9]:
def text_preprocess(res):
    match = re.search(r'(\d+\.\d+|\d+)\s*(\w+)', res)

    if match:
        quantity = match.group(1)
        unit = match.group(2).lower()
        print(unit)
        prediction.append(f"{quantity} {unit}")
        print(f"{quantity} {unit}")
#         return f"{quantity} {unit}"
    
    else:
        prediction.append("")
        print("No match found")

In [10]:
def llm_text(entity_name):
    image = Image.open('/kaggle/working/img.jpg').convert('RGB')
    
    question = f"Analyse the given image and give me a the {entity_name} from the image and give the answere in this form The {entity_name} value from the image is"
#     question = f'Analyse the image and give this from the image {entity_name}'
    msgs = [{'role': 'user', 'content': question}]

    res = model.chat(
        image=image,
        msgs=msgs,
        tokenizer=tokenizer,
        sampling=True,
        temperature=0.7,
        system_prompt='You are an AI that extracts and transcribes text from images. Please read the text in the image and transcribe it accurately.'
    )
    
    return res

In [11]:
# Your allowed unit map
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon', 'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Define your list
input_list = ['20 cm', '45 grams', '7 v']

# Function to convert units
def convert_units(input_list, entity_unit_map):
    converted_list = []
    
    # Unit conversion map (short form to long form)
    unit_conversion = {
        'cm': 'centimetre',
        'cms': 'centimetre',
        'centimeters': 'centimetre',
        'centimetr': 'centimetre',
        'gram': 'gram',
        'grams': 'gram',
        'g': 'gram',
        'kg': 'kilogram',
        'kilograms': 'kilogram',
        'lb': 'pound',
        'lbs': 'pound',
        'ounces': 'ounce',
        'oz': 'ounce',
        'ft': 'foot',
        'feet': 'foot',
        'in': 'inch',
        'inches': 'inch',
        'v': 'volt',
        'volts': 'volt',
        'voltage': 'volt',
        'mm': 'millimetre',
        'millimeters': 'millimetre',
        'ml': 'millilitre',
        'milliliters': 'millilitre',
        'l': 'litre',
        'liters': 'litre'
    }
    
    for item in input_list:
        # Skip empty strings
        if not item.strip():
            converted_list.append(item)
            continue
        
        # Split the value and the unit
        value, unit = item.split()
        
        # Convert the unit if it's in the unit_conversion map
        if unit in unit_conversion:
            long_unit = unit_conversion[unit]
            converted_list.append(f"{value} {long_unit}")
        else:
            converted_list.append(item)  # No conversion needed, append as is
    
    return converted_list




In [12]:
prediction = []
index = []

In [13]:

for i in tqdm(range(10000, 13000)):
    index.append(data['index'][i])
    download_image(data['image_link'][i], image_path)
    entity = data['entity_name'][i]
#     print(entity)
    res1 = llm_text(entity)
#     print(res1)
    text_preprocess(res1)

  0%|          | 0/3000 [00:00<?, ?it/s]

Downloaded: /kaggle/working/img.jpg


  0%|          | 1/3000 [00:07<6:27:07,  7.75s/it]

mm
70 mm
Downloaded: /kaggle/working/img.jpg


  0%|          | 2/3000 [00:11<4:35:56,  5.52s/it]

...

w
300 w
Downloaded: /kaggle/working/img.jpg


100%|█████████▉| 2999/3000 [5:37:27<00:06,  6.42s/it]

No match found
Downloaded: /kaggle/working/img.jpg


100%|██████████| 3000/3000 [5:37:36<00:00,  6.75s/it]

No match found


In [14]:
# ensure prediction and index length
len(prediction)
len(index)

3000

In [15]:
# Convert the units
converted_list = convert_units(prediction, entity_unit_map)

print(converted_list)

['70 millimetre', '120 millimetre', '70 millimetre', '2.7 inch', '2.7 inch', '40 millimetre', '40 millimetre', '', '4.0 centimetre', '', '21 centimetre', '21 centimetre', '', '9.1 inch', '3.7 volt', '330 pound', '5.5 pound', '5 to', '', '110 volt', '2.2 pound', '66 millimetre', '8 ounce', '14.1 ounce', '24.01 inch', '63.5 centimetre', '63.5 centimetre', '3 to', '3.8 inch', '10.9 inch', '', '30 kilogram', '', '4.4 ounce', '400 pound', '123 millimetre', '123 millimetre', '15.0 centimetre', '15.0 centimetre', '12.1 centimetre', '12.1 centimetre', '27.9 centimetre', '2 7', '18 volt', '', '8.25 inch', '5.82 inch', '56 gram', '18.5 centimetre', '13.6 inch', '16 centimetre', '8.6 inch', '8.6 inch', '150 kilogram', '', '5 volt', '35 millimetre', '35 millimetre', '9.5 inch', '9.5 inch', '9.5 inch', '32 centimetre', '84 gram', '1 pound', '', '700 millimetre', '80 gram', '58 centimetre', '58 centimetre', '', '6 ounce', '50 sheets', '210 millimetre', '120 volt', '4.5 centimetre', '9 inch', '9 inch

In [16]:
# Ensure converted list length
len(converted_list)

3000

In [17]:
data = {'index': index, 'prediction': converted_list}
df = pd.DataFrame(data)


In [18]:
df.to_csv('test_out.csv', index=False)  # Disables the index when saving to a file
